# Определение стоимости автомобилей

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В вашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Вам нужно построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

### Признаки:

•	DateCrawled — дата скачивания анкеты из базы

•	VehicleType — тип автомобильного кузова

•	RegistrationYear — год регистрации автомобиля

•	Gearbox — тип коробки передач

•	Power — мощность (л. с.)

•	Model — модель автомобиля

•	Kilometer — пробег (км)

•	RegistrationMonth — месяц регистрации автомобиля

•	FuelType — тип топлива

•	Brand — марка автомобиля

•	Repaired — была машина в ремонте или нет

•	DateCreated — дата создания анкеты

•	NumberOfPictures — количество фотографий автомобиля

•	PostalCode — почтовый индекс владельца анкеты (пользователя)

•	LastSeen — дата последней активности пользователя

**Целевой признак**
Price — цена (евро)


## Подготовка данных

In [1]:
pip install lightgbm

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install phik

Note: you may need to restart the kernel to use updated packages.


In [3]:
# Импортируем все необходимое
import pandas as pd
import numpy as np
import seaborn as sns
import scipy.stats as st
import matplotlib.pyplot as plt
import time
import phik
from phik import resources, report
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV, train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression
import lightgbm as lgb


# Отключаем предупреждения
import warnings
warnings.filterwarnings('ignore')

# Откроем файл и выведем первые 5 строк
try:
    df = pd.read_csv('https://code.s3.yandex.net/datasets/autos.csv')
except:
    df = pd.read_csv('autos.csv')
    
df.head()

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,Repaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Kilometer          354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  Repaired           283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

In [5]:
df.describe()

,Price,RegistrationYear,Power,Kilometer,RegistrationMonth,NumberOfPictures,PostalCode
count,354369.000000,354369.000000,354369.000000,354369.000000,354369.000000,354369.0,354369.000000
mean,4416.656776,2004.234448,110.094337,128211.172535,5.714645,0.0,50508.689087
std,4514.158514,90.227958,189.850405,37905.341530,3.726421,0.0,25783.096248
min,0.000000,1000.000000,0.000000,5000.000000,0.000000,0.0,1067.000000
25%,1050.000000,1999.000000,69.000000,125000.000000,3.000000,0.0,30165.000000
50%,2700.000000,2003.000000,105.000000,150000.000000,6.000000,0.0,49413.000000
75%,6400.000000,2008.000000,143.000000,150000.000000,9.000000,0.0,71083.000000
max,20000.000000,9999.000000,20000.000000,150000.000000,12.000000,0.0,99998.000000


In [6]:
df.dtypes

DateCrawled          object
Price                 int64
VehicleType          object
RegistrationYear      int64
Gearbox              object
Power                 int64
Model                object
Kilometer             int64
RegistrationMonth     int64
FuelType             object
Brand                object
Repaired             object
DateCreated          object
NumberOfPictures      int64
PostalCode            int64
LastSeen             object
dtype: object

In [7]:
df.nunique()

DateCrawled          271174
Price                  3731
VehicleType               8
RegistrationYear        151
Gearbox                   2
Power                   712
Model                   250
Kilometer                13
RegistrationMonth        13
FuelType                  7
Brand                    40
Repaired                  2
DateCreated             109
NumberOfPictures          1
PostalCode             8143
LastSeen             179150
dtype: int64

In [9]:
df.columns = (df.columns.str.replace('(?<=[a-z])(?=[A-Z])', '_', regex=True).str.lower())
df.columns

Index(['date_crawled', 'price', 'vehicle_type', 'registration_year', 'gearbox',
       'power', 'model', 'kilometer', 'registration_month', 'fuel_type',
       'brand', 'repaired', 'date_created', 'number_of_pictures',
       'postal_code', 'last_seen'],
      dtype='object')

In [10]:
# Выведем колличество пропущенных значений для каждого столбца
df.isna().sum()

date_crawled              0
price                     0
vehicle_type          37490
registration_year         0
gearbox               19833
power                     0
model                 19705
kilometer                 0
registration_month        0
fuel_type             32895
brand                     0
repaired              71154
date_created              0
number_of_pictures        0
postal_code               0
last_seen                 0
dtype: int64

In [11]:
df.duplicated().sum()

4

Удалим дубликаты

In [12]:
df = df.drop_duplicates()

Удалим неинформативные признаки

In [13]:
df.drop(['date_crawled', 'model', 'registration_month', 'date_created',
         'number_of_pictures', 'postal_code', 'last_seen'], axis=1, inplace=True)

Разделим признаки на категориальные и количественные.
Посмотрим на распределение в оставшихся признаках.

In [14]:
categorical_columns = ['vehicle_type', 'gearbox', 'fuel_type', 'brand', 'repaired']
numerical_columns = ['price', 'registration_year', 'power', 'kilometer']

In [15]:
for column in categorical_columns:
    print(df[column].value_counts())

sedan          91457
small          79830
wagon          65165
bus            28775
convertible    20203
coupe          16161
suv            11996
other           3288
Name: vehicle_type, dtype: int64
manual    268249
auto       66283
Name: gearbox, dtype: int64
petrol      216349
gasoline     98719
lpg           5310
cng            565
hybrid         233
other          204
electric        90
Name: fuel_type, dtype: int64
volkswagen        77012
opel              39931
bmw               36914
mercedes_benz     32044
audi              29456
ford              25179
renault           17927
peugeot           10998
fiat               9643
seat               6907
mazda              5615
skoda              5500
smart              5246
citroen            5148
nissan             4941
toyota             4606
hyundai            3587
sonstige_autos     3374
volvo              3210
mini               3202
mitsubishi         3022
honda              2817
kia                2465
suzuki             232

In [16]:
for column in numerical_columns:
    print(df[column].value_counts())

0        10772
500       5670
1500      5394
1000      4648
1200      4594
         ...  
1368         1
233          1
11080        1
16340        1
10985        1
Name: price, Length: 3731, dtype: int64
2000    24490
1999    22727
2005    22109
2001    20123
2006    19900
        ...  
6500        1
2290        1
1001        1
9450        1
3500        1
Name: registration_year, Length: 151, dtype: int64
0       40225
75      24023
60      15897
150     14590
101     13298
        ...  
610         1
6006        1
5815        1
6018        1
1241        1
Name: power, Length: 712, dtype: int64
150000    238207
125000     36453
100000     14882
90000      11567
80000      10047
70000       8593
60000       7444
5000        6397
50000       6232
40000       4911
30000       4436
20000       3974
10000       1222
Name: kilometer, dtype: int64


In [17]:
for column in categorical_columns:
    df[column].fillna(df[column].mode()[0], inplace=True)

In [18]:
df = df[df['price'] > 1]

In [19]:
power = df[(df['power'] >= 1) & (df['power'] <= 1000)]
power_median = power['power'].median()

df.loc[(df['power'] < 1) | (df['power'] > 1000), 'power'] = power_median


In [20]:
df = df[(df['registration_year'] >= 1970) & (df['registration_year'] <= 2016)]

In [21]:
df.nunique()

price                3688
vehicle_type            8
registration_year      47
gearbox                 2
power                 474
kilometer              13
fuel_type               7
brand                  40
repaired                2
dtype: int64

In [22]:
df.isnull().sum()

price                0
vehicle_type         0
registration_year    0
gearbox              0
power                0
kilometer            0
fuel_type            0
brand                0
repaired             0
dtype: int64

#### Вывод: 

1. Было найдено 4 дубликата. Мы их удалили.

2. Столбцы с неинформативными для моделей признаками были удалены.

3. Пропуски были обнаружены только в категориальных признаках, их мы заполнили модой.

4. Убрали данный с нулевой ценой.

5. Ограничили разброс объявлений по мощности автомобилей. 

6. Обработаны аномалии в столбце RegistrationYear. Мы оставили временной промежуток с 1970 по 2016 год. Автомобили до 1970 года являются раритетными и не подходят для автосервиса заказчика, а обьявления после 2016го года могут быть не действительными так как данные предоставлены до 2016го года. Недействительные годы регистрации мы заменили медианой действительных годов регистрации (используем медиану так как она менее чувствительна к выбросам).

## Обучение моделей

In [23]:
RANDOM_STATE = np.random.RandomState(12345)

In [24]:
# Разделим данные на обучающие и тестовые выборки
train_df, test_df = train_test_split(df, test_size=0.2, random_state=RANDOM_STATE)

In [25]:
# Кодирование для обучающих и тестовых выборок
train_df_encoded = pd.get_dummies(train_df, drop_first=True)
test_df_encoded = pd.get_dummies(test_df, drop_first=True)

In [26]:
# Общие колонки в выборках
set_columns = set(train_df_encoded.columns) & set(
    test_df_encoded.columns)

train_df_encoded = train_df_encoded[list(set_columns)]
test_df_encoded = test_df_encoded[list(set_columns)]


In [27]:
# Разделим тестовую и тренировочную выборки на признаки X и целевой признак Y
x_train = train_df_encoded.drop('price', axis=1)
y_train = train_df_encoded['price']

x_test = test_df_encoded.drop('price', axis=1)
y_test = test_df_encoded['price']

In [28]:
x_train.shape

(262036, 57)

In [29]:
y_train.shape

(262036,)

In [30]:
x_test.shape

(65510, 57)

In [31]:
y_test.shape

(65510,)

Создадим списки чтобы проанализировать время обучения, прогнозирования и качество моделей с помощью библиотеки time. Сравним значения RMSE.

In [32]:
train_time = []
prediction_time = []
cv_rmse = []

#### Линейная регрессия

In [33]:
# Масштабируем данные для линейной регрессии
scaler = StandardScaler()

x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

In [34]:
lr = LinearRegression()

# Диапазоны гиперпараметров для перебора
param_lr = {
    'fit_intercept': [True, False],
    'copy_X': [True, False]
}

# Используем RandomizedSearchCV для поиска по диапазону гиперпараметров и выбора лучшей модели
lr_search = RandomizedSearchCV(estimator = lr,
                               param_distributions = param_lr,
                               n_iter = 10,
                               cv = 5,
                               refit = True,
                               n_jobs = 1)

lr_search.fit(x_train_scaled, y_train)

# Результаты поиска лучшей модели
print("Лучшие параметры для модели lr: ", lr_search.best_params_)

Лучшие параметры для модели lr:  {'fit_intercept': True, 'copy_X': True}


In [35]:
lr = LinearRegression()


start_time = time.time()
lr.fit(x_train_scaled, y_train)
train_time.append(time.time() - start_time)

start_time = time.time()
lr.predict(x_train_scaled)
prediction_time.append(time.time() - start_time)

cvs = cross_val_score(LinearRegression(
    fit_intercept = True,
    copy_X = True),
                      x_train_scaled,
                      y_train, scoring='neg_mean_squared_error',
                      cv=5,
                      n_jobs=-1)

cv_rmse.append(np.sqrt(-np.mean(cvs)))

print('Training Time:', train_time)
print('Prediction Time:', prediction_time)
print('RMSE:', cv_rmse)

Training Time: [0.43631672859191895]
Prediction Time: [0.026005983352661133]
RMSE: [2803.972026917503]


#### Модель LightGBM

In [36]:
df.head()

,price,vehicle_type,registration_year,gearbox,power,kilometer,fuel_type,brand,repaired
0,480,sedan,1993,manual,110,150000,petrol,volkswagen,no
1,18300,coupe,2011,manual,190,125000,gasoline,audi,yes
2,9800,suv,2004,auto,163,125000,gasoline,jeep,no
3,1500,small,2001,manual,75,150000,petrol,volkswagen,no
4,3600,small,2008,manual,69,90000,gasoline,skoda,no


In [37]:
categorical_feature = [1, 3, 6, 7, 8]

In [38]:
lgbm = lgb.LGBMRegressor(random_state=RANDOM_STATE, categorical_feature = categorical_feature)

# Диапазоны гиперпараметров для перебора
lgbm_params = {
    'learning_rate': [0.01, 0.05, 0.1],
    'n_estimators': [50, 100, 200],
    'num_leaves': [20, 50, 100],
    'min_data_in_leaf': [20, 30, 50]
}

# Используем RandomizedSearchCV для поиска по диапазону гиперпараметров и выбора лучшей модели
lgbm_random_search = RandomizedSearchCV( lgbm, lgbm_params,
                                        n_iter=20,
                                        scoring='neg_mean_squared_error',
                                        verbose=2,
                                        random_state=RANDOM_STATE, 
                                        cv=5,
                                        n_jobs=-1,
                                       )

lgbm_random_search.fit(x_train_scaled, y_train)

best_lgbm = lgbm_random_search.best_estimator_

train_time.append(lgbm_random_search.refit_time_)
cv_rmse.append(np.sqrt(-lgbm_random_search.best_score_))

start_time = time.time()
best_lgbm.predict(x_train_scaled)
prediction_time.append(time.time() - start_time)

print('Training Time:', train_time)
print('Prediction Time:', prediction_time)
print('RMSE:', cv_rmse)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30
Training Time: [0.43631672859191895, 1.2634053230285645]
Prediction Time: [0.026005983352661133, 0.3140709400177002]
RMSE: [2803.972026917503, 1664.3520015769116]


In [39]:
data = {
    'Модель': ['Линейная регрессия', 'LightGBM'],
    'Время обучения': train_time,
    'Время предсказания': prediction_time,
    'RMSE': cv_rmse
}


general_table = pd.DataFrame(data)
print(general_table)

               Модель  Время обучения  Время предсказания         RMSE
0  Линейная регрессия        0.436317            0.026006  2803.972027
1            LightGBM        1.263405            0.314071  1664.352002


#### Вывод.

Наилучшие результаты показала модель LightGBM. 

Время обучения: 1.70812  


Время предсказания: 0.426148 


RMSE: 1691.780048

## Тестирование лучшей модели

In [40]:
test_prediction = best_lgbm.predict(x_test_scaled)
test_rmse = np.sqrt(mean_squared_error(y_test, test_prediction))

print('RMSE для лучшей модели LightGBM:', test_rmse)

RMSE для лучшей модели LightGBM: 1658.8407134920908


Модель LightGBM на тестовых данных показала результат RMSE = 1693.6525273925465. Уровень метрики соответствует требованиям, значит цель достугнута.

## Вывод

Целью нашего исследования было создать модель для предсказания стоимости автомобилей на основе представленных данных. 

1. Было найдено 4 дубликата. Мы их удалили. Столбцы с неинформативными для моделей признаками были удалены. Пропуски были обнаружены только в категориальных признаках, их мы заполнили модой. Убрали данный с нулевой ценой. Ограничили разброс объявлений по мощности автомобилей. Обработаны аномалии в столбце RegistrationYear. Мы оставили временной промежуток с 1970 по 2016 год. Недействительные годы регистрации мы заменили медианой действительных годов регистрации.

2. Были обучены две модели: модель линейной регрессии и модель LightGBM. Лучший результат показала модель LightGBM.

Время обучения: 1.70812 

Время предсказания: 0.426148

RMSE: 1691.780048

3. На тестовых данных модель LightGBM показала следующие результаты:

RMSE = 1693.6525273925465

Этот результат соответствует требованиям и целям нашего исследования.